# Evaluation of SIFT positional transformation uncertatinty
0) Specify datasets

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from sift_uncertainty import *

scenes = ['Alamo', 'Ellis_Island', 'Madrid_Metropolis', 'NYC_Library', 'Piazza_del_Popolo', 'Roman_Forum', 'Tower_of_London', 'Union_Square', 'Vienna_Cathedral', 'Yorkminster']
scales = [3.97745, 0.94967, 14.44161, 6.95844, 6.25074, 24.38904, 15.72399, 6.85125, 15.07437, 13.26471]

1) Load all the keypoints (angles, scales, positions) and GT homographies

In [ ]:
data = {'sift_a1':[], 'sift_a2': [], 'sift_s1': [], 'sift_s2': [], 'sift_u1': [], 'sift_u2': [], 'gt_H1to2': [], 'gt_Hids': []}
for scene, scale in zip(scenes, scales):
    data = collect_all(scene, scale, data)

2. Compute the squared mean reprojection error for each keypoint pair, i.e.,
$\epsilon_{x_i} = \sqrt{(|x_i - \widetilde{H}_i x_i'|_2^2 + |x_i' - \widetilde{H}_i^{-1} x_i|_2^2)/8}$, where the factor 8 guaranteeing that $\epsilon_{x_i}$ can be compared to the expected uncertainty of the coordinates.

In [ ]:
eps_x = np.zeros(len(data['sift_a1']),)
num_inliers = np.zeros(len(data['sift_a1']),)

H2pts_ids = [[] for j in range(len(data['gt_H1to2']))]
for j,x in enumerate(data['gt_Hids']):
    H2pts_ids[x].append(j)

for i in tqdm(range(len(data['gt_H1to2']))):
    ids = H2pts_ids[i]
    H = data['gt_H1to2'][i]
    u1 = np.array([data['sift_u1'][j] for j in ids])
    u2 = np.array([data['sift_u2'][j] for j in ids])
    num_inliers[ids] = np.ones((len(ids),)) * len(ids)
    eps_1to2 = get_translation_error(u1, u2, H)
    eps_2to1 = get_translation_error(u2, u1, np.linalg.inv(H))
    eps_x[ids] = np.sqrt((eps_1to2**2 + eps_2to1**2)/8)

std_eps = np.sqrt(np.sum(eps_x**2)/len(eps_x))
print(f"The standard deviation of the squared mean reprojection error is {std_eps:.3f}px.")

3) Show the histogram of the squared mean reprojection error $\epsilon_{x}$

In [ ]:
plt.hist(eps_x, 100, range=[0,6])
plt.xlabel('The squared mean reprojection error $\epsilon_{x_i}$')
plt.ylabel('Occurrence')
plt.title('Histogram of the squared mean reprojection error')
plt.yscale('log')
plt.savefig('histogram_of_robust_squared_mean_reprojection_error2.png', dpi=100)
plt.show()

4) Plot the heatmap of the squared mean reprojection error for chosen scale clusters 

In [ ]:
from sift_uncertainty import *
cluster1_mean = 1.5
clusterN_mean = 9.5
N_clusters = 9
min_samples_for_cluster = 20
outlier_threshold = 4

clusters_mean = np.linspace(cluster1_mean, clusterN_mean, N_clusters)
inliers_ids = [j for j,x in enumerate(eps_x) if x<outlier_threshold]
eps_x_filtered = np.array(eps_x)[inliers_ids]
s1_filtered = np.array(data['sift_s1'])[inliers_ids]
s2_filtered = np.array(data['sift_s2'])[inliers_ids]
num_inliers_filtered = np.array(num_inliers)[inliers_ids]
eps_grid, eps_grid_robust = cluster_reprojection_errors(s1_filtered, s2_filtered, \
    eps_x_filtered, num_inliers_filtered, clusters_mean, N_clusters)
eps_std, eps_std_robust = positional_statistics_for_bins(eps_grid, eps_grid_robust, N_clusters, min_samples_for_cluster)
plot_heatmap(np.flip(eps_std,axis=0), clusters_mean, np.flip(clusters_mean), "The std of squared mean reprojection error [px]", \
    "sift scale in the first image", "sift scale in the second image", "std_of_squared_mean_reprojection_error.png")

plot_heatmap(np.flip(eps_std_robust,axis=0), clusters_mean, np.flip(clusters_mean), "The robust std of squared mean reprojection error [px]", \
    "sift scale in the first image", "sift scale in the second image", "std_of_robust_squared_mean_reprojection_error.png")